# Cargo librerías



In [2]:
import numpy as np #manejo de arreglos

import matplotlib.pyplot as plt #gráficos

from sklearn.decomposition import PCA #Componentes principales
from sklearn.preprocessing import StandardScaler #Escalado de datos

from sklearn.linear_model import LogisticRegression #regresión logística

# matriz de confusión: https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# métricas para evaluar la clasificación predicha
from sklearn.metrics import accuracy_score, recall_score, precision_score
# otras métricas se pueden ver acá: https://scikit-learn.org/stable/modules/model_evaluation.html#

# divide en conjuntos para entrenar y para testeat
from sklearn.model_selection import train_test_split

# Tamaño de gráficos
plt.rcParams["figure.figsize"] = (8,8)

# Levanto datos, train/test split, evaluaciones.

In [3]:
# Levanto los datos
datos = np.loadtxt('./heart.csv',delimiter=',', skiprows=1)#, usecols = (0,2,3,4,5,6,7,8) )
nombres = ['age','sex','cp','rbs','sc','fbs120','recr','mhr','eia','op','slope','ca','thal']

# extraigo variables (predictoras, features, características, son todos nombres que se usan)
X = datos[:,0:-1]
# extraigo clasificación (target, labels, etiquetas, son todos nombres que se usan)
y = datos[:,-1]


# Separo en train/test. El stratify es para conservar el balance de clases.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=474077)

#IMPORTANTE: sigue estando el problema que el split influye mucho en la predicción (recordar encuentro virtual con nros. primos). 
# Para resolver, o mejor dicho amortiguar este efecto, se hace Cross Validation (Validación Cruzada), que por ahora queda en el tintero.
# La solución de compromiso que usamos es fijar el random_state para hacer comparables las métricas entre diferentes corridas.
# De todos modos, puede pensar por su cuenta alguna estrategia para amortiguar el efecto que vimos de la gran variabilidad
# que tiene la predicción dependiendo del split aleatorio que se use.


# Standarizo datos de ENTRENAMIENTO
scaler = StandardScaler()
scaler.fit(X_train) #calcula promedio y desvío
# print("Promedio datos entrenamiento", scaler.mean_)
# print("Varianza datos entrenamiento", scaler.var_)
X_train = scaler.transform(X_train) #estandariza con promedio y desvío anteriores

# Standarizo datos de TESTEO con los mismos promedio y desvío calculados para entrenamiento
X_test = scaler.transform(X_test)
# ¿Por qué hago esto? Porque en teoría los datos de entrenamientos son desconocidos a la hora de entrenar el modelo.



lg = LogisticRegression() #instancio la clase
#entreno modelo predictivo a partir de los datos de ENTRENAMIENTO
modelo_lg = lg.fit(X_train, y_train)


# clasifico según el modelo, es decir predigo las clases, con los datos de TESTEO.
# por defecto clasifica por probabilidad más alta (o sea punto de corte 0.5)
y_pred_test = modelo_lg.predict(X_test) 


# métricas de evaluación sobre los datos de TESTEO.
accuracy = accuracy_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)

print('Métricas sobre datos nuevos de TEST')
print('Accuracy: ', round(accuracy,2))
print('Recall: ', round(recall,2))
print('Precision: ', round(precision,2))



# predigo probabilidades según el modelo con los datos de TESTEO.
# Luego uso estas probabilidades para, mediante punto de corte, clasificar en clases.
probas = modelo_lg.predict_proba(X_test)

# por ejemplo, por defecto asumo que SÍ tiene riesgo cardíco (y=1)
y_pred_test_custom = np.ones(y_test.shape)

#y si la probabilidad de y=0 es mayor a 0.8, lo clasifico como sin riesgo (y=0)
for i in range(probas.shape[0]):
    if (probas[i,0]>0.8):
        y_pred_test_custom[i]=0.


#métricas de evaluación sobre los datos de TESTEO, punto de corte arbitrario
accuracy_custom = accuracy_score(y_test, y_pred_test_custom)
recall_custom = recall_score(y_test, y_pred_test_custom)
precision_custom = precision_score(y_test, y_pred_test_custom)

print('')
print('Métricas sobre datos nuevos de TEST, punto de corte arbitrario')
print('Accuracy: ', round(accuracy_custom,2))
print('Recall: ', round(recall_custom,2))
print('Precision: ', round(precision_custom,2))

Métricas sobre datos nuevos de TEST
Accuracy:  0.85
Recall:  0.9
Precision:  0.83

Métricas sobre datos nuevos de TEST, punto de corte arbitrario
Accuracy:  0.78
Recall:  0.92
Precision:  0.74
